In [6]:
search_for = 116
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.00874781608581543
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 476884191
type: [1, 1, 1, 1, 116] 116
cases of this type: 181063936
10000 43.63132491547568
20000 40.16399020556716
30000 46.54426438998282
40000 42.09156821164802
50000 194.5768309815764
60000 220.53049802453063
70000 200.2395224243486
80000 206.18595318982193
90000 156.41964951202425
100000 178.10140098790427
110000 176.17065337232307
120000 196.59414307659262
130000 189.9966703728449
140000 188.69531974461145
150000 192.20801296077508
160000 233.62850146805164
170000 217.944444841637
180000 289.5747825091716
190000 314.97906708967145
200000 371.9741044052264
210000 334.3896402873273
220000 356.6873608207851
230000 338.2474566045721
240000 359.80512215116534
250000 332.458499092818
260000 346.06099230384086
270000 327.0006023620741
280000 349.42497869675327
290000 331.607505473789
300000 353.2799805441864
310000 335.15428944538

3080000 390.85478246189837
3090000 128.79620165760147
3100000 107.65436036632164
3110000 105.43489815585913
3120000 108.84386402429028
3130000 89.99205080923232
3140000 87.36127867976701
3150000 84.89200315571686
3160000 85.23402723973726
3170000 83.35242637111762
3180000 86.6393631897322
3190000 83.62407634090194
3200000 477.68100225026234
3210000 370.341601675116
3220000 379.4007314601378
3230000 306.07845687785453
3240000 359.31593290973643
3250000 352.2764502323241
3260000 372.5257505363661
3270000 370.2442068019198
3280000 376.06635735193265
3290000 379.38923789491287
3300000 364.3276958095266
3310000 375.3848258506947
3320000 361.7707092153307
3330000 378.253428159623
3340000 343.06277045389714
3350000 385.82492281625065
3360000 339.79651629367146
3370000 371.83112729355486
3380000 329.1332725136205
3390000 368.3408148118026
3400000 339.26070666341025
3410000 355.78389422185575
3420000 355.7301701258836
3430000 366.27001412694324
3440000 371.8809757147109
3450000 354.324343874633

6190000 105.12199206535504
6200000 105.43849428916295
6210000 105.26712062780729
6220000 105.63584225309913
6230000 106.8689412721332
6240000 104.94917393520863
6250000 291.2449433426241
6260000 360.9309504271988
6270000 406.7126424323667
6280000 85.20878676677847
6290000 85.26334960337788
6300000 83.09777531582108
6310000 84.1006130511895
6320000 82.66118853767591
6330000 85.4602018968637
6340000 142.42486217376307
6350000 441.6036054642705
6360000 338.8908653866073
6370000 360.5017967599743
6380000 295.32242657037153
6390000 383.72494052478703
6400000 355.86520086569084
6410000 365.36908982081616
6420000 327.1452100885462
6430000 373.95634381982813
6440000 327.88246215286006
6450000 353.38193628790214
6460000 325.9424046732172
6470000 363.43415079481764
6480000 333.10405270720315
6490000 353.558853785478
6500000 335.88236186769444
6510000 363.74898095425624
6520000 331.5866855677344
6530000 344.83965120308284
6540000 347.1634111183369
6550000 344.96804002000687
6560000 336.3304503972

9300000 101.32019772400297
9310000 105.44544951794182
9320000 101.99736002954124
9330000 106.01905370444823
9340000 136.07768570610648
9350000 453.9468178108913
9360000 393.82200854404994
9370000 411.5514629675764
9380000 308.62909861339165
9390000 347.7855551876837
9400000 335.1094180424606
9410000 413.6539428162613
9420000 357.4425063559097
9430000 83.35226617018344
9440000 85.09129113219224
9450000 86.47054979310899
9460000 83.53708127143034
9470000 82.52715007618626
9480000 81.92187115778542
9490000 193.7591563491915
9500000 409.67846801765234
9510000 365.33955915965225
9520000 356.30113426609216
9530000 319.78769503105923
9540000 346.19132611550805
9550000 344.92845896914406
9560000 345.80765282706085
9570000 329.3004767774808
9580000 353.412543550468
9590000 342.02475448015076
9600000 343.85660359401606
9610000 314.3276865000208
9620000 356.12661138425085
9630000 304.0152745130732
9640000 325.44326098125543
9650000 315.82157525830365
9660000 329.4749623551902
9670000 331.41980191

12320000 312.0854507122996
12330000 349.09019852176544
12340000 324.68212636333595
12350000 353.02845397554285
12360000 351.7309166962289
12370000 287.25372261111045
12380000 103.00627513169587
12390000 104.73113249414212
12400000 101.94228842371683
12410000 104.6358045128616
12420000 102.32851717616242
12430000 103.37093415998625
12440000 326.998960093409
12450000 391.70913088054243
12460000 363.0606602087654
12470000 309.5738994966427
12480000 362.1933533452335
12490000 342.6891289270139
12500000 351.3496354929348
12510000 321.6743403271104
12520000 351.2279238778539
12530000 314.989645774348
12540000 338.98892933290864
12550000 325.8790631407197
12560000 402.71396466841423
12570000 331.18474185019
12580000 82.20309569553697
12590000 84.09058618594091
12600000 84.59533834993586
12610000 82.47647500560088
12620000 82.37932441828963
12630000 81.80362652824091
12640000 248.5031436354166
12650000 387.2299051601916
12660000 381.33769111042216
12670000 309.432332945418
12680000 345.8694588

...

464980000 4.213764462516983
464990000 4.024082679585264
465000000 3.055160810142537
465010000 0.9476778090384205
465020000 3.154570243132988
465030000 3.622409103756904
465040000 3.7255975242351953
465050000 4.455738719867282
465060000 4.118720698209286
465070000 3.480337319433649
465080000 3.581202049860835
465090000 2.64661595787253
465100000 1.8276763737129769
465110000 2.232117483823101
465120000 2.6035417539356946
465130000 3.5445487414075005
465140000 3.435968267599205
465150000 3.925800074923873
465160000 3.959672721257123
465170000 3.7869831781408982
465180000 3.984880673984786
465190000 3.9151747771629095
465200000 1.5472226240264508
465210000 1.2150504354658525
465220000 4.06744785921532
465230000 4.009841491070012
465240000 3.6010585731514695
465250000 3.939681098795089
465260000 4.180978590790867
465270000 3.953972971009612
465280000 3.8626427141296196
465290000 1.9957640955209666
465300000 1.6631679943117101
465310000 2.208762721862303
465320000 3.606019756975081
465330000

467830000 2.466151651346094
467840000 2.572478543442276
467850000 2.235865427516103
467860000 2.1950638708058263
467870000 1.5829999244842994
467880000 2.0082459032321176
467890000 2.0168042423650556
467900000 2.26025092258715
467910000 1.8700964551343124
467920000 1.0667773572892547
467930000 0.634351904334267
467940000 0.7517645229092836
467950000 0.7960202348153856
467960000 1.1438187631236847
467970000 2.562665767820716
467980000 1.9083019640500414
467990000 2.276078188632495
468000000 1.5944949117800793
468010000 1.7563434845957493
468020000 1.6444760253091255
468030000 1.9901955902695656
468040000 0.6059902283241285
468050000 0.667494214366893
468060000 0.4946551867436767
468070000 0.4291619193677505
468080000 0.36291000129317574
468090000 0.636462854085048
468100000 0.6869283541165724
468110000 0.6690784069759449
468120000 0.6536114289864897
468130000 0.4943090755850937
468140000 0.5618419441612098
468150000 0.6539392656860153
468160000 0.6610691665041514
468170000 0.64289700164

470670000 0.2950949792501529
470680000 0.41102600855711435
470690000 0.8240013978261351
470700000 1.4955929321716033
470710000 0.8578104274036684
470720000 0.8387286866114472
470730000 1.737296055657506
470740000 0.9740055386189355
470750000 0.7291905001350376
470760000 1.8617726815150974
470770000 0.8152026206926836
470780000 0.7978996199715404
470790000 1.1593636927924356
470800000 1.5791421149645777
470810000 1.053456795387255
470820000 1.473830932631433
470830000 0.846935184591492
470840000 1.5805787288194033
470850000 0.30601034585036835
470860000 0.3983102649769121
470870000 0.5131395936940114
470880000 0.8017011375752885
470890000 0.9253519462353123
470900000 0.774321074142966
470910000 0.9002136062451005
470920000 0.7494010844539073
470930000 1.2900666644380954
470940000 1.2172173951538403
470950000 1.4006150861958795
470960000 0.8321447919971346
470970000 0.9818408299604654
470980000 1.1035349010204543
470990000 0.9551720110005206
471000000 0.7526476000438929
471010000 1.09199

473410000 0.965288802263644
473420000 0.6133798498399986
473430000 0.6322680788707137
473440000 0.31918243031658733
473450000 0.21580912754332357
473460000 0.5201794465039771
473470000 0.6945376721855401
473480000 0.8678272311091423
473490000 0.8559844017486772
473500000 0.8087223025002016
473510000 0.759392117035614
473520000 0.5749985951986909
473530000 0.3218864542836481
473540000 0.2994944243950049
473550000 0.2902065068205396
473560000 0.45728435501845016
473570000 0.7413832542538709
473580000 0.73743075658981
473590000 0.5743266132110623
473600000 0.6631324475316537
473610000 0.6605108759764433
473620000 0.6725685000521806
473630000 0.5084356197132535
473640000 0.26958000722076497
473650000 0.41009250794776947
473660000 0.41052400008892354
473670000 0.6911597985493542
473680000 0.755520124958972
473690000 0.7460803272022803
473700000 0.732226981583655
473710000 0.41267649477163293
473720000 0.2810468409776886
473730000 0.394273248318553
473740000 0.24455695127216978
473750000 0.5

476080000 0.07983402133809196
476090000 0.22269957564508255
best so far: 0
type: [2, 2, 1, 29, 1] 116
cases of this type: 48778
476100000 0.1419805558938384
476110000 0.051024832994328605
476120000 0.04021435227362314
476130000 0.027694870786070823
476140000 0.03748513561906417
best so far: 0
type: [2, 2, 29, 1, 1] 116
cases of this type: 1682
476150000 0.043488828944729434
best so far: 0
type: [2, 29, 1, 1, 2] 116
cases of this type: 464
best so far: 0
type: [2, 29, 1, 2, 1] 116
cases of this type: 232
best so far: 0
type: [2, 29, 2, 1, 1] 116
cases of this type: 116
best so far: 0
type: [2, 58, 1, 1, 1] 116
cases of this type: 58
best so far: 0
type: [4, 1, 1, 1, 29] 116
cases of this type: 707281
476160000 0.20363172149684033
476170000 0.15255977745714452
476180000 0.17599367471490304
476190000 0.13885752398220697
476200000 0.09390902974551253
476210000 0.10600288998887936
476220000 0.06783268477594853
476230000 0.07077008325341012
476240000 0.06614336096930504
476250000 0.123059056